https://realpython.com/python-metaclasses/

The term metaprogramming refers to the potential for a program to have knowledge of or manipulate itself. Python supports a form of metaprogramming for classes called metaclasses.

Metaclasses are an esoteric OOP concept, lurking behind virtually all Python code. You are using them whether you are aware of it or not. For the most part, you don’t need to be aware of it. Most Python programmers rarely, if ever, have to think about metaclasses.

New-Style Classes

New-style classes unify the concepts of class and type. If obj is an instance of a new-style class, type(obj) is the same as obj.__class__:

In [1]:
class Foo:
    pass
obj= Foo()
obj.__class__, type(obj), obj.__class__ is type(obj)

(__main__.Foo, __main__.Foo, True)

In [5]:
n = 5
d = { 'x' : 1, 'y' : 2 }

class Foo:
     pass

x = Foo()

for obj in (n, d, x):
     print(type(obj) is obj.__class__)
     print(type(obj), obj.__class__)

True
<class 'int'> <class 'int'>
True
<class 'dict'> <class 'dict'>
True
<class '__main__.Foo'> <class '__main__.Foo'>


Remember that, in Python, everything is an object. Classes are objects as well. As a result, a class must have a type. What is the type of a class?

Consider the following:

In [6]:
class Foo:
    pass
x= Foo()
type(x), type(Foo)

(__main__.Foo, type)

The type of x is class Foo, as you would expect. But the type of Foo, the class itself, is type. In general, the type of any new-style class is type.

The type of the built-in classes you are familiar with is also type:

In [7]:
for t in int, float, dict, list, tuple:
...     print(type(t))

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


In [8]:
type(type)

type

type is a metaclass, of which classes are instances. Just as an ordinary object is an instance of a class, any new-style class in Python, and thus any class in Python 3, is an instance of the type metaclass.

In the above case:

    x is an instance of class Foo.
    Foo is an instance of the type metaclass.
    type is also an instance of the type metaclass, so it is an instance of itself.


You can also call type() with three arguments—type(<name>, <bases>, <dct>):

    <name> specifies the class name. This becomes the __name__ attribute of the class.
    <bases> specifies a tuple of the base classes from which the class inherits. This becomes the __bases__ attribute of the class.
    <dct> specifies a namespace dictionary containing definitions for the class body. This becomes the __dict__ attribute of the class.

Calling type() in this manner creates a new instance of the type metaclass. In other words, it dynamically creates a new class.

In each of the following examples, the top snippet defines a class dynamically with type(), while the snippet below it defines the class the usual way, with the class statement. In each case, the two snippets are functionally equivalent.

## Example 1

In this first example, the <bases> and <dct> arguments passed to type() are both empty. No inheritance from any parent class is specified, and nothing is initially placed in the namespace dictionary. This is the simplest class definition possible:

In [9]:
>>> Foo = type('Foo', (), {})

>>> x = Foo()
>>> x

In [10]:
>>> class Foo:
...     pass
...
>>> x = Foo()
>>> x

## Example 2

Here, <bases> is a tuple with a single element Foo, specifying the parent class that Bar inherits from. An attribute, attr, is initially placed into the namespace dictionary:

In [13]:
>>> Bar = type('Bar', (Foo,), dict(attr=100))

>>> x = Bar()

>>> x.attr, x.__class__, x.__class__.__bases__


(100, __main__.Bar, (__main__.Foo,))

In [14]:
>>> class Bar(Foo):
...     attr = 100
...

>>> x = Bar()
>>> x.attr, x.__class__, x.__class__.__bases__

(100, __main__.Bar, (__main__.Foo,))

## Custom Metaclasses

Consider again this well-worn example:

In [15]:
>>> class Foo:
...     pass
...
>>> f = Foo()


The expression Foo() creates a new instance of class Foo. When the interpreter encounters Foo(), the following occurs:

    The __call__() method of Foo’s parent class is called. Since Foo is a standard new-style class, its parent class is the type metaclass, so type’s __call__() method is invoked.

    That __call__() method in turn invokes the following:
        __new__()
        __init__()

If Foo does not define __new__() and __init__(), default methods are inherited from Foo’s ancestry. But if Foo does define these methods, they override those from the ancestry, which allows for customized behavior when instantiating Foo.

In the following, a custom method called new() is defined and assigned as the __new__() method for Foo:

In [16]:
>>> def new(cls):
...     x = object.__new__(cls)
...     x.attr = 100
...     return x
...
>>> Foo.__new__ = new

>>> f = Foo()
>>> f.attr

100

This modifies the instantiation behavior of class Foo: each time an instance of Foo is created, by default it is initialized with an attribute called attr, which has a value of 100. (Code like this would more usually appear in the __init__() method and not typically in __new__(). This example is contrived for demonstration purposes.)

Now, as has already been reiterated, classes are objects too. Suppose you wanted to similarly customize instantiation behavior when creating a class like Foo. If you were to follow the pattern above, you’d again define a custom method and assign it as the __new__() method for the class of which Foo is an instance. Foo is an instance of the type metaclass, so the code looks something like this:

In [17]:
# Spoiler alert:  This doesn't work!
>>> def new(cls):
...     x = type.__new__(cls)
...     x.attr = 100
...     return x
...
>>> type.__new__ = new

TypeError: can't set attributes of built-in/extension type 'type'

Except, as you can see, you can’t reassign the __new__() method of the type metaclass. Python doesn’t allow it.

This is probably just as well. type is the metaclass from which all new-style classes are derived. You really shouldn’t be mucking around with it anyway. But then what recourse is there, if you want to customize instantiation of a class?

One possible solution is a custom metaclass. Essentially, instead of mucking around with the type metaclass, you can define your own metaclass, which derives from type, and then you can muck around with that instead.

The first step is to define a metaclass that derives from type, as follows:

In [18]:
>>> class Meta(type):
...     def __new__(cls, name, bases, dct):
...         x = super().__new__(cls, name, bases, dct)
...         x.attr = 100
...         return x

The definition header class Meta(type): specifies that Meta derives from type. Since type is a metaclass, that makes Meta a metaclass as well.

Note that a custom __new__() method has been defined for Meta. It wasn’t possible to do that to the type metaclass directly. The __new__() method does the following:

    Delegates via super() to the __new__() method of the parent metaclass (type) to actually create a new class
    Assigns the custom attribute attr to the class, with a value of 100
    Returns the newly created class

Now the other half of the voodoo: Define a new class Foo and specify that its metaclass is the custom metaclass Meta, rather than the standard metaclass type. This is done using the metaclass keyword in the class definition as follows:

In [19]:
>>> class Foo(metaclass=Meta):
...     pass
...
>>> Foo.attr

100

In the same way that a class functions as a template for the creation of objects, a metaclass functions as a template for the creation of classes. Metaclasses are sometimes referred to as class factories.

## Is This Really Necessary?

As simple as the above class factory example is, it is the essence of how metaclasses work. They allow customization of class instantiation.

Still, this is a lot of fuss just to bestow the custom attribute attr on each newly created class. Do you really need a metaclass just for that?

In Python, there are at least a couple other ways in which effectively the same thing can be accomplished:

Simple inheritance

In [20]:
>>> class Base:
...     attr = 100
...

>>> class X(Base):
...     pass
>>> X.attr

100

Class Decorator:

In [21]:
>>> def decorator(cls):
...     class NewClass(cls):
...         attr = 100
...     return NewClass
...
>>> @decorator
... class X:
...     pass
>>> X.attr

100